<img style="width:100%;" src="http://labs.jstor.org/content/images/2020/03/Screen-Shot-2020-03-27-at-3.01.38-PM.jpg">
<h1 style="text-align:center;background-color:black;"> FeedBack Comepetition <i> [BASE-line + Visualization] </i></h1>

<h3> Given Information: </h3>

**id - ID code for essay response <br>
discourse_id - ID code for discourse element <br>
discourse_start - character position where discourse element begins in the essay response <br>
discourse_end - character position where discourse element ends in the essay response <br>
discourse_text - text of discourse element <br>
discourse_type - classification of discourse element <br>
discourse_type_num - enumerated class label of discourse element <br>
predictionstring - the word indices of the training sample, as required for predictions<br>**

**Language models like NER implementation are ideal for this case**

<h1 style="font-family: sans-seriff; padding: 5px;"><mark> ALWAYS THINK BEFORE DOING </mark></h1>

### Extraction part

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
import shutil
import statsmodels
import os
import tensorflow as tf
import string
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn 
import time
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/feedback-prize-2021/train.csv')
df_train.index = df_train.id

path = '../input/feedback-prize-2021/'

train_data = []
train_id = []
test_data = []
test_id = []

for name in os.listdir(path + 'test'):
    with open(path + 'test/' + name, 'r', encoding='latin-1') as f:
        test_data.append(f.read())
        test_id.append(name[:-4])
        
for name in os.listdir(path + 'train'):
    with open(path + 'train/' + name, 'r', encoding='latin-1') as f:
        train_data.append(f.read())
        train_id.append(name[:-4])

df_train.predictionstring = df_train.predictionstring.agg(lambda k: k.split(' '))
df_train['word_start'] = df_train.predictionstring.agg(lambda k: int(k[0]))
df_train['word_end'] = df_train.predictionstring.agg(lambda k: int(k[-1]))
df_train = df_train.drop(['id', 'discourse_id','discourse_start', 'discourse_end'], axis =1)
display(df_train)

In [ ]:
punc = string.punctuation
new_punc = punc.replace("~", "") + '.'
print('New_Punc: ', new_punc)
train_text_corpus = '~~~'.join(train_data)
train_text_corpus = train_text_corpus.translate(str.maketrans('','',new_punc))
train_data = train_text_corpus.split('~~~')
df_text = pd.DataFrame(train_data, columns = ['train_text'], index=train_id)

In [ ]:
''' Loading & Saving File '''

# import pickle as pkl

# dir_path_obj = open('./df_train.txt', 'wb')
# pkl.dump(df_train, dir_path_obj)
# dir_path_obj.close()

# dir_path_obj = open('./df_text.txt', 'wb')
# pkl.dump(df_text, dir_path_obj)
# dir_path_obj.close()


In [ ]:
def use_fn(k):
    k_type = [ele for k_each in k for ele in k_each]
    return k_type

df_plot = df_train.groupby(['id','discourse_type'])['predictionstring'].agg(use_fn)
all_ids = list(set(df_train.index))

In [ ]:
lmt = 5
sns.set_style('darkgrid')
fig, axs = plt.subplots(lmt, 1 , figsize=(25,15))
# tests_id = '62C57C524CD2'
print('--'*20,'Visual-Repersentation', '--'*20)
for num, tests_id in enumerate(all_ids[:lmt]):
    for step, name in enumerate(df_plot[tests_id].keys()):
        y_axs = np.ones(len((df_text.loc[tests_id].train_text).split(' '))) * 7
        x_axs = np.arange(0,len((df_text.loc[tests_id].train_text).split(' ')),1)
        value = df_plot[tests_id][name]
        axs[num].set_title(tests_id)
        if step==0:
            axs[num].plot(x_axs, y_axs, 'ko', label='Text')
        axs[num].plot(np.array(value, dtype='int32'), np.ones(len(value))*step, label=name, marker='o')
        axs[num].legend()    
#     plt.show()
plt.tight_layout(pad=2)
plt.show()
    

In [ ]:
first_time = time.time()
check_id = '423A1CA112E2'
sample_dict = {}
classes = list(set(df_train.discourse_type.values))
for sample in all_ids:
    each_dict = {name:0.0 for name in classes}
    for name in (df_plot[sample].keys()):        
        each_dict[name] = len(df_plot[sample][name]) / len((df_text.loc[tests_id].train_text).split(' '))
        sample_dict[sample] = each_dict
        
second_time = time.time()
print('Time_Taken: ', second_time-first_time)
print('sample_dict_Done')    

In [ ]:
first_time = time.time()
train_token = tf.keras.preprocessing.text.Tokenizer(split = ' ', lower=True)
train_token.fit_on_texts([' '.join(df_text.train_text.values.tolist())])
x_train = np.array(train_token.texts_to_sequences(df_text.train_text.values))
second_time = time.time()
word_vocab = train_token.word_index
print('time_taken: ', second_time - first_time)

In [ ]:
y_train = np.array([list(list(sample_dict.values())[i].values()) for i in range(len(all_ids))]) # all the proba_distrib of the multi-label statements
print('x_train_shape : ', x_train.shape)
print('y_train_shape : ', y_train.shape)

In [ ]:
max_len = len(x_train[0])
for i in x_train:
    if len(i) > max_len:
        max_len = len(i)
print('MAX_LEN: ', max_len)     
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len, padding='post')
print('x_train_shape: ', x_train.shape)
count = 0

In [ ]:
### Use tf.data.Dataset
from sklearn.model_selection import train_test_split
def splitter(x_train, y_train):
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, random_state=28, test_size = 0.15)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=28)
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape ,x_test.shape, y_test.shape)
    return x_train, y_train, x_val, y_val, x_test, y_test

if count == 0:
    x_train, y_train, x_val, y_val, x_test, y_test = splitter(x_train, y_train)

count += 1

# take a batch_size thats multiple to the train_sample_size
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(28)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(28)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(28)

##### Try to Match the Truth Probabability Distribution using KL divergence

In [ ]:
# print(x_train.shape[0], x_val.shape[0], x_test.shape[0])
df_distrib = df_train['discourse_type'].value_counts()
plt.title('_Distrib_Class_')
df_distrib.plot(figsize=(12,6), kind='bar')
plt.show()

<b><mark> We create sample/Class weights but wont use them in these notebook  </mark></b><br>
As this is a Experimental/Exploratory/Baseline NoteBook implementation

In [ ]:
total_arr = np.ones(len(df_train))
start_idx = 0
for idx, key in enumerate(df_distrib.keys()):
    total_arr[start_idx : start_idx + df_distrib[key]] = idx
    start_idx = start_idx + df_distrib[key]
    
print(total_arr.shape)

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(total_arr), total_arr)
class_weights = dict(enumerate(class_weights))
sample_weights = np.array([class_weights.get(_ele) for _ele in total_arr])

sample_new_weights = np.ones(shape = (len(np.unique(df_train.index)), len(list(class_weights))))
for row in range(sample_new_weights.shape[0]):
    for col in range(sample_new_weights.shape[-1]):
        sample_new_weights[int(row)][int(col)] = class_weights.get(col)
        
print(sample_new_weights.shape)

### MIMIC MODEL 

##### Only way using sample_weight with sequential generation | using a decoder model to decode the encoder states 

In [ ]:
''' Train MIMIC-MODEL '''
# print('The MImic Model')

mimic_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(x_train.shape[-1])),
    tf.keras.layers.Embedding(input_dim=len(word_vocab), output_dim = 64),
    tf.keras.layers.GRU(32, return_sequences=True, recurrent_dropout = 0.5, dropout=0.4),
    tf.keras.layers.GRU(32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(classes), activation='softmax'),
    
])

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
mimic_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
mimic_model.summary()
train_ft = time.time()
mimic_model.fit(x_train, y_train, epochs=5, validation_data=val_dataset, callbacks=[es], steps_per_epoch=128)#sample_weight = sample_new_weights[:len(x_train)] )
train_lt = time.time()
print('Time taken for training MIMIC-Model: ', train_lt - train_ft)
mimic_model.save('./' + 'mimic_model.h5')

In [ ]:
mimic_model = tf.keras.models.load_model('../input/models/mimic_model.h5')

In [ ]:
# y_train[0].shape
for seed in range(1):    
    print('--' * 20)
    print('SEED_INDEX: ', seed)
    print()
    print('ground_truth: ',y_test[seed], len(y_test[seed]))
    print('Prediction: ', mimic_model.predict(x_test[seed].reshape(1,-1)))
    print()
    print('--'*20)
mimic_model.evaluate(x_test, y_test)

In [ ]:
print('y_test: ', y_test[seed])
y_pred = mimic_model.predict(x_test[seed].reshape(1,-1))[0]
print('y_pred: ', y_pred)

### Custom Model

In [ ]:
''' MIMICv2-MODEL '''

''' @ Two-Methods :
    1. Retrain the mimic_model (Trained) Model
    2. Train the mimCv2 model independently
'''

class mimCv2(tf.keras.models.Model):
    def __init__(self, input_shape):
        super().__init__()
#         self.build((None,) + input_shape)
        self.mimic_arch = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(input_shape)),
            tf.keras.layers.Embedding(input_dim=len(word_vocab), output_dim = 128),
            tf.keras.layers.GRU(64, return_sequences=True),
            tf.keras.layers.GRU(64),
            tf.keras.layers.Dense(len(classes), activation='softmax')
        ])
        print('__mimCv2_initialized__')
                
        
    def compile(self, mimic_loss, std_loss ,optimizer, metrics, alpha = 0.2, smooth = 10.0):
        super().compile(optimizer = optimizer, metrics = metrics) #calls the parent 'Model' compile method
        self.alpha = alpha
        self.mimic_loss = mimic_loss
        self.std_loss = std_loss
        self.smooth = 10.0
#         steps_per_execution = 128
        
    def train_step(self, train_dataset):
#         print('IN TRAIN STEP')
        x, y = train_dataset
        with tf.GradientTape() as tape:
            mimic_arch_op = self.mimic_arch(x, training=True)
            mimic_arch_op = mimic_arch_op / tf.reduce_sum(mimic_arch_op, axis=0) 
            
            ''' Another Approach maybe softmax-ing outputs '''
#             print(type(mimic_arch_op))
#             mimic_arch_op = tf.nn.softmax(mimic_arch_op)
#             mimic_arch = self(x, training=True)
#             mimic_loss = self.mimic_loss(y / self.smooth, mimic_arch_op / self.smooth)
            '''
            we can use sample_weight while calculating KLDivergence
            '''
            mimic_loss = self.mimic_loss(y , mimic_arch_op)
            std_loss = self.std_loss(y, mimic_arch_op)
            loss =  self.alpha*std_loss + (1 - self.alpha) * mimic_loss 
        train_weights = self.mimic_arch.trainable_variables
        mimic_grad = tape.gradient(loss, train_weights)
        self.optimizer.apply_gradients(zip(mimic_grad, train_weights))
        self.compiled_metrics.update_state(y, mimic_arch_op)
        results = {mtr.name: mtr.result() for mtr in self.metrics}
        results.update({'mimic_loss': mimic_loss})
        return results
    
    def test_step(self, test_dataset):
        print()
#         print('IN TEST STEP')
        x_test, y_test = test_dataset
        mimic_arch_op = self.mimic_arch(x_test, training=False)
        mimic_arch_op = mimic_arch_op / tf.reduce_sum(mimic_arch_op, axis=0) 
#         mimic_arch_op = tf.nn.softmax(mimic_arch_op)
#         mimic_arch_op = self(x_test, training=False)
        mimic_loss = self.mimic_loss(y_test, mimic_arch_op)
        std_loss = self.std_loss(y_test, mimic_arch_op)
        loss = self.alpha * std_loss + (1 - self.alpha) * mimic_loss
        self.compiled_metrics.update_state(y_test, mimic_arch_op)
        results = {mtr.name:mtr.result() for mtr in self.metrics}
        results.update({'mimic_loss': mimic_loss})
        return results
    
    def call(self, x_sample):
        pred_sample = self.mimic_arch(x_sample, training=False)
        return pred_sample
    
input_shape = (x_train.shape[-1],)
mim2_model = mimCv2(input_shape)

kwargs = {
    'mimic_loss' : tf.keras.losses.KLDivergence(),
    'std_loss' : tf.keras.losses.categorical_crossentropy
}
mim2_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2), #, clipnorm=1.0),
                  metrics = [tf.keras.metrics.Accuracy()],
                  **kwargs)

''' Metrics not changing due to less change in loss value | immidiete convergence '''
# mim2_model.build(input_shape)
mim2_model.compute_output_shape(input_shape = (None, )+ input_shape)
mim2_model.summary()
train_ft = time.time()
mim2_model.fit(train_dataset, epochs=3, validation_data=val_dataset, steps_per_epoch=None)#, callbacks=[es])
train_lt = time.time()
print('Time taken for training mimCv2: ', train_lt - train_ft)
print('EVAULATION')
mim2_model.evaluate(test_dataset, steps=20)                
# use shutil/zipfile to create zips of the subclass model directory, and to save the model dir
shutil.make_archive('mim2_model', 'zip', mim2_model.save('./mim2_model'))        

In [ ]:
mim2_model = tf.keras.models.load_model('../input/models/mim2_model')

<b> Final Inference </b> | <i><mark> Not very Good Results ! </mark></i>

In [ ]:
print('__mimCv2_MODEL__')
for seed in range(1):    
    pred = np.ravel(mim2_model.predict(x_test[seed].reshape(1,-1)))
    pred_dict = {key:value for key, value in zip(classes, pred)}
    truth_dict = {key: value for key, value in zip(classes, y_test[seed])}
    
    print('PRED: ', end='\n\n')
    pprint(pred_dict)
    print()
    print('TRUTH: ', end='\n\n')
    pprint(truth_dict)
    print()

print('--' * 20)
print()

print('__MIMIC_MODEL__')
for seed in range(1):
    pred = np.ravel(mimic_model.predict(x_test[seed].reshape(1,-1)))
    pred_dict = {key:value for key, value in zip(classes, pred)}
    truth_dict = {key: value for key, value in zip(classes, y_test[seed])}

    pprint(pred_dict)
    print('PRED: ', end='\n\n')
    print()
    pprint(pred_dict)
    print('TRUTH: ', end='\n\n')
    print()
    pprint(truth_dict)
    print()


### SESSION - STOPPER 

In [ ]:
for i in range(10):
    print('time: ',i)
    time.sleep(30*60)

we can use this information for further analysis and as features to ...
PEACE OUT 